# Putting Wine Data into MongoDB

In [ ]:
%matplotlib inline

import datetime
import json
import pandas
import pymongo
import requests
import seaborn
import time

## Creating a class for the CellarWatch API

In [ ]:
class CellarWatch(object):
    base_url = "https://www.cellar-watch.com"
    cookies = None
    headers = {
        "Accept": "application/json, text/javascript, */*",
        "X-Requested-With": "XMLHttpRequest",
        "Accept-Encoding": "gzip, deflate, sdch, br",
        "Accept-Language": "en-GB,en;q=0.8"
    }
    
    def __init__(self, cookies=None):
        self.cookies = cookies
    
    def _construct_get(self, endpoint, url_components):
        url = "/".join([self.base_url, endpoint])
        args = []
        for k, v in url_components.items():
            args.append("{k}={v}".format(k=k, v=v))
        return "{url}?{args}".format(url=url, args="&".join(args))
    
    def _get(self, url):
        resp = requests.get(url, headers=self.headers, cookies=self.cookies)
        rc = str(resp.status_code)
        if rc.startswith("4") or rc.startswith("5"):
            raise requests.exceptions.HTTPError("Got {rc}, expected 200.".format(rc=rc))
        else:
            return resp.json()
    
    def get_wine_price_history(self, lwin, vintage, up_to=None, name=None):
        """
        Args:
            lwin    (int): The Liv-Ex wine reference number (LWIN).
            vintage (int): The year the wine was harvested.
            up_to   (int): Milliseconds since 1970-01-01.
        Returns:
            dict: A dictionary containing price history and auction information.
        """
        def __tidy_up_block(block):
            wine_data = {
                "_id": "{lwin}-{vintage}".format(lwin=lwin, vintage=vintage),
                "name": name,
                "vintage": str(vintage),
                "lwin": str(lwin),
                "history": []
            }
            for data in block:
                wine_data["history"].append({
                    "date": datetime.datetime.fromtimestamp(data["date"] / 1000),
                    "price": data["value"]
                })
            return wine_data
        
        if up_to is None:
            up_to = int(time.time() * 1000)
            
        url = self._construct_get("chart/individualwinechartpage.do", {
            "_": int(time.time() * 1000),
            "ajaxReq": 1,
            "lwin": lwin,
            "vintage": vintage,
            "type": "max",
            "endTime": up_to
        })
        
        price_history = self._get(url)
        hist = { "auction": None, "market": None, "list": None }
        for block in price_history:
            if "name" not in block or "data" not in block:
                continue
            b = __tidy_up_block(block["data"])
            if "Auction" in block["name"]:
                hist["auction"] = b
            elif "Market" in block["name"]:
                hist["market"] = b
            elif "List" in block["name"]:
                hist["list"] = b
        
        return hist
    
    def get_lwins(self, name):
        """
        Args:
            name (str): The name of the wine (or vineyard) to search for. Alphanumeric only.
        Returns:
            list: A list of matching wines (with corresponding LWINs).
        """
        url = self._construct_get("autocompletewinenames.do",
            {
                "ajaxReq": 1,
                "term": name.replace(" ", "+")
            }
        )
        
        return sorted(self._get(url), key=lambda v: v["id"])

## Set up the CellarWatch API

In [ ]:
# Define the wines and years we want
years = range(2004, 2016)
wines = [
    {
        "name": "Haut Brion",
        "lwin": 1011247
    },
    {
        "name": "Lafite Rothschild",
        "lwin": 1011872
    },
    {
        "name": "Latour",
        "lwin": 1012316
    },
    {
        "name": "Margaux",
        "lwin": 1012781
    },
    {
        "name": "Mouton Rothschild",
        "lwin": 1013544
    }
]

# Copy the "JSESSIONID" cookie from a session created in a browser
cookies = {
    "JSESSIONID": "E1D046B292536855E91389C50D28397F"
}
c = CellarWatch(cookies=cookies)

# Create MongoDB client, set up database
client = pymongo.MongoClient("localhost:27017")
db = client.wine_prices

## Get data, load into MongoDB

In [ ]:
for year in years:
    for wine in wines:
        hist = c.get_wine_price_history(lwin=wine["lwin"], vintage=year, name=wine["name"])
        
        auction_hist = hist["auction"]
        market_hist = hist["market"]
        list_hist = hist["list"]
        
        if auction_hist is not None:
            db.auction_prices.insert_one(auction_hist)
        if market_hist is not None:
            db.market_prices.insert_one(market_hist)
        if list_hist is not None:
            db.list_prices.insert_one(list_hist)

## Explore data in MongoDB

In [ ]:
def prices_list_to_records(wines):
    """
    Converts nested JSON objects to CSV-style records. Duplicated information everywhere.
    """
    for wine in wines:
        for date_price in wine["history"]:
            w = dict(wine)
            w.pop("history", None)
            w["date"] = date_price["date"]
            w["price"] = date_price["price"]
            yield w

collection = db.list_prices

wines = []
for i in collection.find({"vintage": "2008"}):
    wines.append(i)
    
wines = prices_list_to_records(wines)
df = pandas.DataFrame.from_dict(wines)

## Plot that shit, son!

In [ ]:
import seaborn as sns

g = sns.FacetGrid(df, hue='name', size=5, aspect=1.5)
g.map(plt.plot, 'date', 'price').add_legend()
g.ax.set(xlabel='Date',
         ylabel='Price',
         title='2005 Vintages')
g.fig.autofmt_xdate()